# Feature importance


In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import copy

%matplotlib inline



In [5]:
df, y = read_data.csv('../Full_Datav5.csv')
df = proc_cat_df(df)
df.head()

NameError: name 'read_data' is not defined

Here we will build a baseline model, in which we will oversample the target feature to ensure balance in its classes. 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X = df.drop('Y', axis = 1)
y = df['Y']
X.head()

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=2019)

#begin oversampling
oversample = pd.concat([X_train,y_train],axis=1)
max_size = oversample['Y'].value_counts().max()
lst = [oversample]

for class_index, group in oversample.groupby('Y'):
        lst.append(group.sample(max_size-len(group), replace=True))
X_train = pd.concat(lst)
y_train=pd.DataFrame.copy(X_train['Y'])
del X_train['Y']

In [ ]:
#ensure that above worked
y_train.value_counts(normalize=True)

In [ ]:
#establish baseline model to tweak
from sklearn.metrics import classification_report, accuracy_score

clf = RandomForestClassifier(n_jobs=-1)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)

print(f"single run validation accuracy score: {accuracy_score(y_val, y_pred)}")
print(classification_report(y_val, y_pred))

In order to try and make a better model, we will try a random search of hyperparameters of a Random Forest to see what values our random search yields. From there, we can then use a narrower Grid search to find our best performing model, which will be our model to help find the best features.

In [ ]:
#try changing the number of trees

clf1 = RandomForestClassifier(n_estimators = 20,n_jobs=-1, oob_score=True)

clf1.fit(X_train, y_train)

y_train_pred1 = clf1.predict(X_train)
y_val_pred1 = clf1.predict(X_val)

print(f"single run training accuracy: {accuracy_score(y_train, y_train_pred1)}")
print(f"single run validation accuracy: {accuracy_score(y_val, y_val_pred1)}")
print(classification_report(y_val, y_pred))

This slightly boosted our score, so let's try more trees. 

In [ ]:
#try changing the number of trees

clf2 = RandomForestClassifier(n_estimators = 80,n_jobs=-1, oob_score=True)

clf2.fit(X_train, y_train)

y_train_pred2 = clf2.predict(X_train)
y_val_pred2 = clf2.predict(X_val)

print(f"single run training accuracy: {accuracy_score(y_train, y_train_pred2)}")
print(f"single run validation accuracy: {accuracy_score(y_val, y_val_pred2)}")
print(classification_report(y_val, y_pred))

This did not help again, so now we will try a Grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

#define number of trees 
n_estimators = [10, 20, 40, 80]

#define min number of samples at each leaf node
min_samples_leaf = [1, 2, 4]

random_grid = {'n_estimators': n_estimators,
               'min_samples_leaf': min_samples_leaf
              }


In [ ]:
#perform the grid seach, using 3-fold cross_validation


clf = RandomForestClassifier()

rf_grid = GridSearchCV(estimator=clf, param_grid=random_grid,
                         scoring = 'precision',
                         cv = 3,
                         n_jobs = -1
                              )

#fit the model
%timeit rf_grid.fit(X_train, y_train)

In [ ]:
rf_grid.best_params_

In [ ]:
#use these parameters for final model

clf_final = RandomForestClassifier(min_samples_leaf= 1, n_estimators= 40,
                                   n_jobs = -1
                                  )

clf_final.fit(X_train, y_train)

In [ ]:
y_pred_final = clf_final.predict(X_val)

print(classification_report(y_val, y_pred_final))
print()

## Feature importance

For this next part, we are going to compare different ways of feature importance measures. Specifically, we'll look at what sklearn calculates, and then we will use two independent techniques: ELI5 library, and Shapley values. Both will be briefly explaned in theory and will be utilized via independent 3rd party libraries. 

To start, we will use sklearn's built in feature importance measures with our fully-built Random Forest model. 

In [ ]:
plt.figure(figsize=(10,8))

feature_imp = pd.Series(clf_final.feature_importances_, index=X_train.columns).sort_values(ascending=False)
feature_imp.plot(kind = 'barh')
plt.title("Feature Importances")
plt.xlabel('Importance score')
plt.ylabel('Feature')

From this chart, we can see that the most important feature in the model is 'PAY_1', the amount one pays in the month before the current month. Then, there are many features that have a similar importance to each other, with a drop off in importance starting with 'PAY_6'. From a wholistic perspective, it seems the only really important feature about a customer is his or her age. The more important features are more so about the bill amounts and the payment amounts. It's all about the money!



HOWEVER, SCIKIT LEARN DOES NOT CALCULATE THESE VALEUS WELL!!!


This, of course, is useful, but one question that the modeler needs to always ask themselves is how much can he or she trust these findings? First off, do these findings make sense? Is age really the only important attributea about a customer? Second of all, we don't know for sure how the values of the features impact the model's thinking. For instance, could it be that the model acutally found that younger customers had a _lower_ chance of defaulting, when in the real world it could be the complete opposite? If this were the case, one would not have complete trust that this model is learning the appropriate patterns in the data; it might be performing well due to false findings.

In order to ensure our models findings, it would be helpful if we could see how the values of the features impact the model's predictions. From this idea we introduce ELI5, an independent library that shows ...

In [ ]:
X_val.values[1]

In [ ]:
column = df.columns.drop('Y')
column


In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

#rf = clf_final.fit(X_train, y_train)
#perm = PermutationImportance(rf).fit(X_val, y_val)
eli5.show_weights(perm, feature_names = list(column), top=27)

From this, we can see that the most important features and their order are in fact different from sklearn's ordering. Why is this?

The difference is in the way that the measure of importance is calculated. sklearn implements feature importance by calculating the decrease in mean impurity. When training a tree, what is computed is how much each feature decreases the weighted impurity in a tree. For a forest, the impurity decrease from each feature can be averaged and the features are ranked according to this measure. However, there are some problems with this method. For one, the model can be biased in selected features with more categories. Secondly, correlated features' importance scores may be incorrect; for two or more correlated features, from the point of view from the model, any of  correlated features can be used as a predictor, with no significant 'preference' between them. However, once one of the correlated features is used, the importance of the other features are reduced, since the level of impurity that the latter correlated features can reduce has, in fact, already been reduced by that first correated feature, resulting in a lower importance score for the latter correlated features.

A more stable way to look at feature importance is to randomly permute the values of the feature and measure the decrease in accuracy between models without  permuted feautre and models with the permuted feature. In the eli5 PermutationImportance class, feature importances are calculated in a similar fashion, which is what the results in the output above show. For each feature, the values are permutated, and the degree in decrease in accuracy with its values permutated is what determines the feature importance score. 


We can check local examples to see how the values of these features affect the model's predictions. We use the INSERT HERE.

In [ ]:
print(f"the model predicted:")
eli5.show_prediction(rf, doc= X_val.values[2019], 
                     show_feature_values = True, 
                     feature_names = list(X_train.columns),
                    )

In [ ]:
# set up random search

from sklearn.model_selection import RandomizedSearchCV

#define number of trees 
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 20)]

#define number of features to use at every split
#max_features = ['auto', .60]

#define max depth in trees
#max_depth = [int(t) for t in np.linspace(start = 10, stop = 100, num = 10)]

#define min number of samples to split a node
#min_samples_split = [2, 5, 8]

#define min number of samples at each leaf node
min_samples_leaf = [1, 2, 4]

#define method of sample selection for each tree
#bootstrap = [True, False]


#create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap
                }



In [ ]:

#perform the random seach, using 3-fold cross_validation
#we'll search over 100 different combinations

clf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator=clf, param_distributions=random_grid,
                              n_iter = 100, cv = 3,
                               random_state = 2019, n_jobs = -1
                              )

#fit the model
rf_random.fit(X, y)



From the UCI dataset website:

This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. This study reviewed the literature and used the following 23 variables as explanatory variables: 
- X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit. 
- X2: Gender (1 = male; 2 = female). 
- X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others). 
- X4: Marital status (1 = married; 2 = single; 3 = others). 
- X5: Age (year). 
- X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: X6 = the repayment status in September, 2005;
- X7 = the repayment status in August, 2005; . . .;X11 = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly; 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
- X12-X17: Amount of bill statement (NT dollar). X12 = amount of bill statement in September, 2005; X13 = amount of bill statement in August, 2005; . . .; X17 = amount of bill statement in April, 2005. 
- X18-X23: Amount of previous payment (NT dollar). X18 = amount paid in September, 2005; X19 = amount paid in August, 2005; . . .;X23 = amount paid in April, 2005. 